In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import sys

In [ ]:
sys.path.append('/home/karthik/code/pyrobo/network_models/')
from multicam_net import PurchasePredNet

In [ ]:
images_folder = '/data/robotics/pick_episodes/'
training_data = [os.path.join(images_folder,x) for x in os.listdir(images_folder)]

In [ ]:

def get_image(feature, name):
    image = feature[name].bytes_list
    arr = np.fromstring(image.value[0],dtype=np.uint8).reshape((480, 640, 3))
    return cv2.resize(arr, (224, 224))


def motor_2_label(motor, direction):
    return 2*motor+(direction+1)/2

def get_total_samples(training_data):
    total_samples = 0
    train_inp = np.zeros((total_samples, ))
    example = tf.train.Example()
    for fl in training_data:
        riter = tf.python_io.tf_record_iterator(fl)
        for dat in riter:
            total_samples+=1
    return total_samples

total_samples = get_total_samples(training_data)


In [ ]:
train_inp = np.zeros((2, total_samples, 224, 224, 3), dtype=np.uint8)
labels = np.zeros((total_samples,), dtype=np.int32)
current_sample = 0
example = tf.train.Example()
for fl in training_data:
    riter = tf.python_io.tf_record_iterator(fl)
    for dat in riter:
        example.ParseFromString(dat)
        feature = example.features.feature
        direction = feature['direction'].int64_list.value[0]
        motor = feature['motor'].int64_list.value[0]
        left_image = get_image(feature, 'left_image')
        right_image = get_image(feature, 'right_image')
        labels[current_sample] = motor_2_label(motor, direction) 
        train_inp[0, current_sample, ...] = left_image
        train_inp[1, current_sample, ...] = right_image
        current_sample+=1


In [ ]:
net = PurchasePredNet()
sess = tf.Session()


In [ ]:
BATCH_SIZE=8

In [ ]:
labels_pl = tf.placeholder(shape=(None,), dtype=tf.int32)

In [ ]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=net.layers['out'], labels=labels_pl))

In [ ]:
optimizer = tf.train.AdadeltaOptimizer(0.001)
train = optimizer.minimize(loss)


In [ ]:
accuracy = tf.metrics.accuracy (labels=labels_pl, predictions=tf.argmax(net.layers['out'], axis=1))


In [ ]:
sess.run(tf.global_variables_initializer())
net.load('/data/image/models/VGG_imagenet.npy', sess, ignore_missing=True)

In [ ]:
for step in range(50000):
    idxes = np.random.randint(0, total_samples-1, size=BATCH_SIZE)
    left_im = train_inp[0, idxes, ...]
    right_im = train_inp[1, idxes, ...]
    labels_x = labels[idxes]
    [_, lv] = sess.run([train, loss], {labels_pl: labels_x, 
                                                    net.left_img:left_im,
                                                    net.right_img: right_im})
    if step%1000 == 0:
        print("Step %d, Loss %f"%(step, lv))

In [ ]:
saver = tf.train.Saver()

In [ ]:
step

In [ ]:
saver.save(sess, '/data/robotics/models/pick_episodes/25000_iters')

In [ ]:
data = np.load('/data/image/models/VGG_imagenet.npy')

In [ ]:
dct = data.item()

In [ ]:
dct['conv5_3']['weights'].shape

In [ ]:
512*3*3*2

In [ ]:
net.layers['merged'].shape

In [ ]:
28*14*512

In [ ]:
sess.close()